# Demand Forecasting with TabPFN

This notebook demonstrates time series forecasting for demand planning using TabPFN's regression capabilities.

**Use Case:** Demand Planning - Forecast product demand by category and region

**Business Context:** Accurate demand forecasting is the foundation of supply chain planning. It drives:
- Inventory planning and safety stock optimization
- Production scheduling and capacity planning
- Distribution requirements planning (DRP)
- Procurement and supplier management

**What you will learn:**
- How to prepare time series data with lag features
- How to use TabPFN for time series forecasting
- How to evaluate forecast accuracy with industry-standard metrics
- How to forecast across multiple series and aggregate for planning

**Prerequisites:** Run `00_data_preparation` notebook first.

## Compute Setup

We recommend running this notebook on **Serverless Compute** with the **Base Environment V4**.

## 1. Installation

In [ ]:
%pip install tabpfn-client scikit-learn pandas matplotlib --quiet

In [ ]:
dbutils.library.restartPython()

## 2. Authentication

In [ ]:
import tabpfn_client

token = dbutils.secrets.get(scope="tabpfn-client", key="token")
tabpfn_client.set_access_token(token)

## 3. Configuration

In [ ]:
CATALOG = "tabpfn_databricks"
SCHEMA = "default"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

## 4. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tabpfn_client import TabPFNRegressor

## 5. Load Demand Forecast Data

The demand forecast dataset contains monthly demand data across:
- Multiple product categories (Beverages, Snacks, Dairy, Frozen, Personal Care)
- Multiple regions (Northeast, Southeast, Midwest, West)

Each series exhibits realistic patterns including:
- Seasonality (annual patterns)
- Trend (growth or decline)
- Holiday effects
- Random noise

In [ ]:
# Load demand forecast data
df_demand = spark.table("demand_forecast").toPandas()
df_demand['date'] = pd.to_datetime(df_demand['date'])

print(f"Dataset shape: {df_demand.shape}")
print(f"Number of time series: {df_demand['series_id'].nunique()}")
print(f"Time range: {df_demand['date'].min().strftime('%Y-%m')} to {df_demand['date'].max().strftime('%Y-%m')}")
print(f"\nCategory distribution:")
print(df_demand['category'].value_counts())

display(df_demand.head(10))

In [ ]:
# Visualize sample time series
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Get one series from each category
categories = df_demand['category'].unique()[:4]

for ax, cat in zip(axes.flatten(), categories):
    series_id = df_demand[df_demand['category'] == cat]['series_id'].iloc[0]
    df_s = df_demand[df_demand['series_id'] == series_id].sort_values('date')
    
    ax.plot(df_s['date'], df_s['demand_units'], 'b-', linewidth=1.5, marker='o', markersize=3)
    ax.set_title(f'{cat} ({df_s["region"].iloc[0]})')
    ax.set_xlabel('Date')
    ax.set_ylabel('Demand (Units)')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Create Lag Features for Time Series Forecasting

To use TabPFN for time series forecasting, we convert the time series problem into a supervised learning problem by creating lag features.

For each time point, we use the previous N months as features to predict the current month's demand.

In [ ]:
def create_lag_features(series, n_lags=12):
    """
    Create lag features for time series forecasting.
    
    Args:
        series: 1D array of time series values
        n_lags: Number of lag features (months of history)
    
    Returns:
        X: Feature matrix of shape (n_samples, n_lags)
        y: Target array of shape (n_samples,)
    """
    X, y = [], []
    for i in range(n_lags, len(series)):
        X.append(series[i-n_lags:i])
        y.append(series[i])
    return np.array(X), np.array(y)


def add_calendar_features(X, dates, n_lags):
    """
    Add calendar features (month, year) to lag features.
    """
    # Convert numpy datetime64 to pandas Timestamp to access month/year
    dates_subset = pd.to_datetime(dates[n_lags:])
    months = np.array([d.month for d in dates_subset])
    years = np.array([d.year for d in dates_subset])
    
    # Cyclical encoding for month
    month_sin = np.sin(2 * np.pi * months / 12)
    month_cos = np.cos(2 * np.pi * months / 12)
    
    X_enhanced = np.column_stack([X, month_sin, month_cos, years - years.min()])
    return X_enhanced

In [ ]:
# Select a single series for detailed demonstration
series_id = df_demand['series_id'].unique()[0]
df_series = df_demand[df_demand['series_id'] == series_id].sort_values('date').reset_index(drop=True)

values = df_series['demand_units'].values
dates = df_series['date'].values

print(f"Selected series: {series_id}")
print(f"Category: {df_series['category'].iloc[0]}")
print(f"Region: {df_series['region'].iloc[0]}")
print(f"Series length: {len(values)} months")

In [ ]:
# Create lag features
n_lags = 12  # Use last 12 months as features
X, y = create_lag_features(values, n_lags)

# Add calendar features
X_enhanced = add_calendar_features(X, dates, n_lags)

print(f"Feature matrix shape: {X_enhanced.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {n_lags} lags + month_sin + month_cos + year_offset")

## 7. Train-Test Split (Time-based)

For time series, we use a time-based split to respect temporal ordering:
- Training: Historical data
- Test: Most recent periods (simulating forecast horizon)

In [ ]:
# Use last 6 months as test set (forecast horizon)
test_size = 6

X_train, X_test = X_enhanced[:-test_size], X_enhanced[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

# Get test dates for visualization
test_dates = dates[n_lags:][-test_size:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)} (forecast horizon: {test_size} months)")

## 8. Train TabPFN Forecaster

In [ ]:
# Train TabPFN regressor
reg = TabPFNRegressor()
reg.fit(X_train, y_train)

# Make predictions
y_pred = reg.predict(X_test)

# Calculate forecast metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Calculate bias
bias = np.mean(y_pred - y_test)
bias_pct = (bias / np.mean(y_test)) * 100

print(f"Forecast Metrics ({test_size}-month horizon):")
print(f"  MAE:  {mae:,.0f} units")
print(f"  RMSE: {rmse:,.0f} units")
print(f"  MAPE: {mape:.1f}%")
print(f"  Bias: {bias:,.0f} units ({bias_pct:+.1f}%)")

In [ ]:
# Visualize forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Full series
all_dates = dates[n_lags:]
ax.plot(all_dates[:-test_size], y_train, 'b-', linewidth=1.5, label='Training Data')

# Actual vs Forecast
ax.plot(test_dates, y_test, 'g-', linewidth=2, marker='o', markersize=8, label='Actual')
ax.plot(test_dates, y_pred, 'r--', linewidth=2, marker='s', markersize=8, label='Forecast')

# Highlight forecast region
ax.axvspan(test_dates[0], test_dates[-1], alpha=0.1, color='yellow', label='Forecast Horizon')

ax.set_xlabel('Date')
ax.set_ylabel('Demand (Units)')
ax.set_title(f'Demand Forecast - {df_series["category"].iloc[0]} ({df_series["region"].iloc[0]}) | MAPE: {mape:.1f}%')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Forecast with Uncertainty Quantification

TabPFN can provide prediction intervals, which is crucial for inventory planning to ensure adequate safety stock.

In [ ]:
# Get predictions with uncertainty (quantiles for 90% prediction interval)
y_lower = reg.predict(X_test, output_type="quantiles", quantiles=[0.05]).flatten()
y_median = reg.predict(X_test, output_type="quantiles", quantiles=[0.5]).flatten()
y_upper = reg.predict(X_test, output_type="quantiles", quantiles=[0.95]).flatten()

# Calculate coverage
coverage = np.mean((y_test >= y_lower) & (y_test <= y_upper))
print(f"90% Prediction Interval Coverage: {coverage:.1%}")

In [ ]:
# Visualize forecast with uncertainty
fig, ax = plt.subplots(figsize=(14, 6))

# Training data
ax.plot(all_dates[:-test_size], y_train, 'b-', linewidth=1.5, label='Training Data', alpha=0.7)

# Forecast with uncertainty band
ax.fill_between(test_dates, y_lower, y_upper, alpha=0.3, color='red', label='90% Prediction Interval')
ax.plot(test_dates, y_median, 'r-', linewidth=2, label='Forecast (Median)')
ax.scatter(test_dates, y_test, color='green', s=100, zorder=5, label='Actual', edgecolors='black')

ax.set_xlabel('Date')
ax.set_ylabel('Demand (Units)')
ax.set_title(f'Demand Forecast with Uncertainty - {df_series["category"].iloc[0]}')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Batch Forecasting Across Multiple Series

In practice, demand planners need to forecast hundreds or thousands of SKU-location combinations. Let's demonstrate batch forecasting.

In [ ]:
# Forecast multiple series
n_series_to_forecast = 10  # Forecast 10 series for demonstration
series_ids = df_demand['series_id'].unique()[:n_series_to_forecast]

results = []
forecast_horizon = 6

print(f"Forecasting {len(series_ids)} series with {forecast_horizon}-month horizon...")
print("="*70)

for sid in series_ids:
    df_s = df_demand[df_demand['series_id'] == sid].sort_values('date')
    vals = df_s['demand_units'].values
    dts = df_s['date'].values
    
    # Create features
    X_s, y_s = create_lag_features(vals, n_lags)
    X_s_enh = add_calendar_features(X_s, dts, n_lags)
    
    # Split
    X_tr, X_te = X_s_enh[:-forecast_horizon], X_s_enh[-forecast_horizon:]
    y_tr, y_te = y_s[:-forecast_horizon], y_s[-forecast_horizon:]
    
    # Train and predict
    model = TabPFNRegressor()
    model.fit(X_tr, y_tr)
    pred = model.predict(X_te)
    
    # Calculate metrics
    mae_s = mean_absolute_error(y_te, pred)
    mape_s = np.mean(np.abs((y_te - pred) / y_te)) * 100
    
    results.append({
        'series_id': sid,
        'category': df_s['category'].iloc[0],
        'region': df_s['region'].iloc[0],
        'MAE': mae_s,
        'MAPE': mape_s,
        'actual_total': y_te.sum(),
        'forecast_total': pred.sum()
    })
    print(f"{sid}: {df_s['category'].iloc[0]:15s} | MAE={mae_s:,.0f}, MAPE={mape_s:.1f}%")

df_results = pd.DataFrame(results)

In [ ]:
# Summary statistics
print("\n" + "="*70)
print("FORECAST ACCURACY SUMMARY")
print("="*70)
print(f"\nOverall Performance:")
print(f"  Average MAPE: {df_results['MAPE'].mean():.1f}%")
print(f"  Median MAPE: {df_results['MAPE'].median():.1f}%")
print(f"  Best MAPE: {df_results['MAPE'].min():.1f}%")
print(f"  Worst MAPE: {df_results['MAPE'].max():.1f}%")

print(f"\nPerformance by Category:")
category_summary = df_results.groupby('category').agg({
    'MAPE': 'mean',
    'series_id': 'count'
}).rename(columns={'series_id': 'n_series'})
print(category_summary.round(1))

In [ ]:
# Visualize MAPE distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MAPE histogram
axes[0].hist(df_results['MAPE'], bins=15, edgecolor='black', alpha=0.7)
axes[0].axvline(df_results['MAPE'].mean(), color='red', linestyle='--', 
                label=f"Mean: {df_results['MAPE'].mean():.1f}%")
axes[0].set_xlabel('MAPE (%)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Forecast Accuracy (MAPE)')
axes[0].legend()

# Actual vs Forecast scatter
axes[1].scatter(df_results['actual_total'], df_results['forecast_total'], 
                alpha=0.7, s=100, c=df_results['category'].astype('category').cat.codes)
max_val = max(df_results['actual_total'].max(), df_results['forecast_total'].max())
axes[1].plot([0, max_val], [0, max_val], 'r--', label='Perfect Forecast')
axes[1].set_xlabel('Actual Total Demand')
axes[1].set_ylabel('Forecast Total Demand')
axes[1].set_title(f'Forecast vs Actual ({forecast_horizon}-month total)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 11. Aggregate Forecasts for Planning

Demand planners often need aggregated forecasts at different levels (category, region, total) for different planning decisions.

In [ ]:
# Aggregate forecasts by category
print("Aggregate Forecast Summary by Category:")
print("="*70)

category_forecast = df_results.groupby('category').agg({
    'actual_total': 'sum',
    'forecast_total': 'sum',
    'series_id': 'count'
}).rename(columns={'series_id': 'n_series'})

category_forecast['forecast_error'] = category_forecast['forecast_total'] - category_forecast['actual_total']
category_forecast['error_pct'] = (category_forecast['forecast_error'] / category_forecast['actual_total']) * 100

print(category_forecast.round(0))

# Total company forecast
total_actual = df_results['actual_total'].sum()
total_forecast = df_results['forecast_total'].sum()
total_error_pct = ((total_forecast - total_actual) / total_actual) * 100

print(f"\nTotal Company Forecast:")
print(f"  Actual: {total_actual:,.0f} units")
print(f"  Forecast: {total_forecast:,.0f} units")
print(f"  Error: {total_error_pct:+.1f}%")

## 12. Business Application: Safety Stock Calculation

The uncertainty quantification from TabPFN can be used to calculate safety stock levels.

In [ ]:
# Calculate safety stock based on forecast uncertainty
# Using the first series as an example

df_s = df_demand[df_demand['series_id'] == series_ids[0]].sort_values('date')
vals = df_s['demand_units'].values
dts = df_s['date'].values

X_s, y_s = create_lag_features(vals, n_lags)
X_s_enh = add_calendar_features(X_s, dts, n_lags)

X_tr, X_te = X_s_enh[:-forecast_horizon], X_s_enh[-forecast_horizon:]
y_tr, y_te = y_s[:-forecast_horizon], y_s[-forecast_horizon:]

model = TabPFNRegressor()
model.fit(X_tr, y_tr)

# Get prediction intervals
p50 = model.predict(X_te, output_type="quantiles", quantiles=[0.5]).flatten()
p95 = model.predict(X_te, output_type="quantiles", quantiles=[0.95]).flatten()

# Safety stock = P95 - P50 (covers upside demand risk)
safety_stock = p95 - p50

print("Safety Stock Calculation (95% service level):")
print("="*60)
print(f"\nSeries: {series_ids[0]}")
print(f"Category: {df_s['category'].iloc[0]}")

ss_df = pd.DataFrame({
    'Month': range(1, forecast_horizon + 1),
    'Forecast (P50)': p50.round(0).astype(int),
    'Upside (P95)': p95.round(0).astype(int),
    'Safety Stock': safety_stock.round(0).astype(int)
})
display(ss_df)

print(f"\nTotal Safety Stock Required: {safety_stock.sum():,.0f} units")
print(f"Safety Stock as % of Forecast: {(safety_stock.sum() / p50.sum()) * 100:.1f}%")

## Summary

In this notebook, we demonstrated:

- ✅ **Time Series Feature Engineering** - Creating lag and calendar features
- ✅ **Single Series Forecasting** - Training TabPFN on historical demand
- ✅ **Uncertainty Quantification** - Prediction intervals for risk assessment
- ✅ **Batch Forecasting** - Forecasting multiple series efficiently
- ✅ **Aggregate Analysis** - Rolling up forecasts for planning
- ✅ **Safety Stock Calculation** - Using uncertainty for inventory planning

**Key Takeaways:**
1. TabPFN can be effectively used for time series forecasting with lag features
2. Built-in uncertainty quantification enables safety stock optimization
3. Batch forecasting allows scaling to enterprise-level SKU counts

**Typical Demand Planning MAPE Benchmarks:**
- Excellent: < 15%
- Good: 15-25%
- Acceptable: 25-35%
- Needs Improvement: > 35%

**Next Steps:**
- Integrate forecasts with inventory planning systems
- Add external features (promotions, holidays, weather)
- Implement hierarchical forecast reconciliation
- Deploy as a scheduled Databricks workflow